# Cómo afinar los modelos de lenguaje grandes con adaptadores LoRA usando Hugging Face TRL

Este notebook demuestra cómo afinar con precisión de manera eficiente modelos de lenguaje grandes usando adaptadores LoRA (adaptación de bajo rango). LoRA es una técnica de fine-tuning con eficiencia de parámetros que:
- Congela los pesos del modelo pre-entrenado.
- Agrega pequeñas matrices de descomposición de rangos entrenables a las capas de atención
- Generalmente reduce los parámetros entrenables en aproximadamente un 90 %
- Mantiene el rendimiento del modelo mientras es eficiente en el uso de la memoria

Cubriremos:
1. Configuración del entorno de desarrollo y configuración de LoRA
2. Creación y preparación del conjunto de datos para el entrenamiento del adaptador
3. _Fine-tuning_ utilizando `trl` y `SFTTrainer` con adaptadores LoRA
4. Prueba del modelo y fusión de adaptadores (opcional)

## 1. Configura el entorno de desarrollo

Nuestro primer paso es instalar las librerías de Hugging Face y PyTorch, incluidas `trl`, `transformers` y `datasets`. Si todavía no sabes qué es `trl`, no te preocupes. Se trata de una librería nueva integrada con `transformers`y `datasets`, que facilita el fine-tuning, aprendizaje por refuerzo o el alineamiento de modelos de lenguaje.


In [ ]:
# Instala los requisitos en Google Colab
# !pip install transformers datasets trl huggingface_hub

# Autenticación en Hugging Face

from huggingface_hub import login

login()

# Por comodidad, puedes crear una variable de entorno que contenga tu token de Hugging Face como HF_TOKEN

## 2. Carga el conjunto de datos

In [13]:
# Carga el conjunto de datos
from datasets import load_dataset

# TODO: define tu conjunto de datos y configuración utilizando los parámetros de ruta y nombre.
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. Fine-tuning del LLM usando `trl` y el `SFTTrainer` con LoRA

El [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) de `trl` está integrado con los adaptadores LoRA mediante la biblioteca [PEFT](https://huggingface.co/docs/peft/en/index). Las ventajas clave de esta configuración incluyen:

1. **Eficiencia de memoria**: 
   - Solo los parámetros del adaptador se almacenan en la memoria de la GPU
   - Los pesos del modelo base permanecen congelados y pueden cargarse en menor precisión
   - Permite el fine-tuning de modelos grandes en GPUs de consumo

2. **Características de entrenamiento**:
   - Integración nativa de PEFT/LoRA con configuración mínima
   - Soporte para QLoRA (LoRA Cuantizado) para una eficiencia de memoria aún mejor

3. **Gestión de los adaptadores**:
   - Guardado de pesos del adaptador durante puntos de control
   - Funciones para fusionar adaptadores de vuelta al modelo base

Usaremos LoRA en nuestro ejemplo, que combina LoRA con cuantización de 4 bits para reducir aún más el uso de memoria sin sacrificar el rendimiento. La configuración requiere solo unos pocos pasos:
1. Definir la configuración de LoRA (rank, alpha, dropout)
2. Crear el SFTTrainer con la configuración PEFT
3. Entrenar y guardar los pesos del adaptador

In [ ]:
# Importa las librerias necesarias
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Carga el modelo y el tokenizador
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Establece el formato de chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Establece el nombre para nuestro fine-tuning que se guardará y/o subirá
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

El SFTTrainer cuenta con una integración nativa con peft, lo que hace que sea muy sencillo ajustar eficientemente los LLMs utilizando, por ejemplo, LoRA. Solo necesitamos crear nuestra LoraConfig y proporcionarla al entrenador. 
<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'> 
    <h2 style='margin: 0;color:blue'>Ejercicio: Define los parámetros de LoRA para fine-tuning</h2> 
    <p>Escoge un conjunto de datos de Hugging Face y realiza fine-tuning a un modelo con él.</p> 
    <p><b>Niveles de dificultad</b></p> 
    <p>🐢 Usa los parámetros generales para un fine-tuning arbitrario</p> 
    <p>🐕 Ajusta los parámetros y revisalos en weights & biases.</p> 
    <p>🦁 Ajusta los parámetros y muestra el cambio en los resultados de inferencia.
    </p> 
</div>

In [ ]:
from peft import LoraConfig

# TODO: Configura los parámetros de LoRA
# r: dimensión de rango para las matrices de actualización de LoRA (más pequeño = más compresión)
rank_dimension = 6
# lora_alpha: factor de escala para las capas LoRA (más alto = adaptación más fuerte)
lora_alpha = 8
# lora_dropout: probabilidad de dropout para las capas LoRA (ayuda a prevenir el sobreajuste)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Dimensión de rango - normalmente entre 4-32
    lora_alpha=lora_alpha,  # Factor de escala LoRA - normalmente 2x el rango
    lora_dropout=lora_dropout,  # Probabilidad de dropout para las capas LoRA
    bias="none",  # Tipo de sesgo para LoRA. los sesgos correspondientes se actualizarán durante el entrenamiento.
    target_modules="all-linear",  # A qué módulos aplicar LoRA
    task_type="CAUSAL_LM",  # Tipo de tarea para la arquitectura del modelo
)

Antes de comenzar nuestro entrenamiento, necesitamos definir los hiperparámetros (TrainingArguments) que queremos utilizar.

In [ ]:
# Configuración del entrenamiento
# Hiperparámetros basados en las recomendaciones del artículo sobre QLoRA
args = SFTConfig(
    # Configuración de salida
    output_dir=finetune_name,  # Directorio para guardar los checkpoints del modelo
    # Duración del entrenamiento
    num_train_epochs=1,  # Número de épocas de entrenamiento
    # Configuración del tamaño de lote
    per_device_train_batch_size=2,  # Tamaño de lote por GPU
    gradient_accumulation_steps=2,  # Acumular gradientes para un lote efectivo más grande
    # Optimización de memoria
    gradient_checkpointing=True,  # Intercambiar cómputo por ahorro de memoria
    # Configuración del optimizador
    optim="adamw_torch_fused",  # Usar AdamW fusionado para mayor eficiencia
    learning_rate=2e-4,  # Tasa de aprendizaje (artículo sobre QLoRA)
    max_grad_norm=0.3,  # Umbral de recorte de gradiente
    # Programación de la tasa de aprendizaje
    warmup_ratio=0.03,  # Porción de pasos para el calentamiento
    lr_scheduler_type="constant",  # Mantener la tasa de aprendizaje constante después del calentamiento
    # Registro y guardado
    logging_steps=10,  # Registrar métricas cada N pasos
    save_strategy="epoch",  # Guardar checkpoint cada época
    # Configuración de precisión
    bf16=True,  # Usar precisión bfloat16
    # Configuración de integración
    push_to_hub=False,  # No subir a HuggingFace
    report_to="none",  # Desactivar registro externo
)

Ahora tenemos todos los componentes básicos que necesitamos para crear nuestro `SFTTrainer` y comenzar a entrenar nuestro modelo.

In [ ]:
max_seq_length = 1512  # longitud máxima de secuencia para el modelo y el empaquetado del conjunto de datos

# Crea SFTTrainer con configuración LoRA
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # Configuración LoRA
    max_seq_length=max_seq_length,  # Longitud máxima de la secuencia
    tokenizer=tokenizer,
    packing=True,  # Habilita el empaquetado de entrada para mayor eficiencia
    dataset_kwargs={
        "add_special_tokens": False,  # Los tokens especiales son manejados por la plantilla
        "append_concat_token": False,  # No se necesita un separador adicional
    },
)

Comienza a entrenar nuestro modelo llamando al método `train()` en nuestra instancia de `Trainer`. Esto iniciará el bucle de entrenamiento y entrenará nuestro modelo durante 3 épocas. Dado que estamos utilizando un método PEFT, solo guardaremos los pesos del modelo adaptado y no el modelo completo.

In [ ]:
# Inicia el entrenamiento, el modelo se guardará automáticamente en Hugging Face y en el directorio de salida
trainer.train()

# Guarda el modelo
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

El entrenamiento con Flash Attention durante 3 épocas con un conjunto de datos de 15k muestras tardó 4:14:36 en un `g5.2xlarge`. La instancia cuesta `1.21$/h`, lo que nos lleva a un coste total de solo ~`5.3$`.

### Fusiona el Adaptador LoRA con el Modelo Original

Cuando se usa LoRA, solo entrenamos los pesos del adaptador mientras mantenemos el modelo base congelado. Durante el entrenamiento, guardamos solo estos pesos de adaptador ligeros (~2-10MB) en lugar de una copia completa del modelo. Sin embargo, para la implementación, es posible que desee volver a fusionar los adaptadores con el modelo base para:

1.  **Implementación Simplificada**: Un solo archivo de modelo en lugar de modelo base + adaptadores.
2.  **Velocidad de Inferencia**: Sin sobrecarga de computación del adaptador.
3.  **Compatibilidad con Frameworks**: Mejor compatibilidad con frameworks de servicio.


In [ ]:
from peft import AutoPeftModelForCausalLM


# Carga el modelo PEFT en CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Fusiona LoRA y el modelo base, y guarda
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. Prueba el Modelo y Ejecuta la Inferencia

Una vez que finaliza el entrenamiento, queremos probar nuestro modelo. Cargaremos diferentes muestras del conjunto de datos original y evaluaremos el modelo en esas muestras, utilizando un bucle simple y la precisión como nuestra métrica.


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Ejercicio Adicional: Carga el Adaptador LoRA</h2>
    <p>Utiliza lo que aprendiste del notebook de ejemplo para cargar tu adaptador LoRA entrenado para la inferencia.</p> 
</div>

In [30]:
# Libera memoria otra vez
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Carga el modelo con el adaptador PEFT
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

Probemos algunas muestras de *prompts* y veamos cómo se desempeña el modelo.

In [34]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)